In [1]:
from dataclasses import dataclass
from pathlib import Path
import torch
import torch.nn as nn
from jaxtyping import Float, Int
import requests
import unicodedata
from collections import Counter


# Training Data

In [2]:
def get_gutenberg_book(
	id: int|None = 84,
	data_temp: Path|str = "../../../../data/gutenberg_data",
	remove_gutenberg_meta: bool = True,
) -> str:
	
	data_temp = Path(data_temp)
	data_temp.mkdir(parents=True, exist_ok=True)
	
	url: str = f"https://www.gutenberg.org/cache/epub/{id}/pg{id}.txt"
	data_path: Path = Path(data_temp) / f"{id}.txt"
	data: str
	# read from cache if it exists
	if data_path.exists():
		with open(data_path, 'r', encoding='utf-8') as file:
			data = file.read()
	else:
		# download if it doesn't exist
		response = requests.get(url)
		response.raise_for_status()  # Ensure that the download was successful
		data = response.text

		# save to cache
		with open(data_path, 'w', encoding='utf-8') as file:
			file.write(data)

	# remove header/footer
	if remove_gutenberg_meta:
		data = '***'.join(data.split('***')[2:])
		data = '***'.join(data.split('***')[:-1])
	
	return data

def get_many_books(
		ids: list[int],
		data_temp: Path|str = "../data/gutenberg_data",
	) -> list[list[str]]:
	
	data: list[str] = []
	for id in ids:
		print(f"Getting book {id}...")
		item: str = get_gutenberg_book(id, data_temp)
		print(f"\t{len(item)} characters read")
		data.append(item)
	
	return data

# Model Definition

In [3]:
@dataclass
class Config():
    d_model: int
    d_vocab: int
    d_hidden: int

In [4]:
class MLP(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.linear1 = nn.Linear(config.d_model, config.d_hidden)
        self.linear2 = nn.Linear(config.d_hidden, config.d_model)
        

    def forward(self, x: Float[torch.Tensor, "seq_len d_model"]) -> Float[torch.Tensor, "seq_len d_model"]:
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
        return x

In [5]:
class AttentionHead(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.config = config
        self.W_qk = nn.Linear(config.d_model, config.d_model)
        self.W_vo = nn.Linear(config.d_model, config.d_model)
        self.softmax = nn.Softmax(dim=-1)
        

    def create_mask(self, n_c: int) -> torch.Tensor:
        mask: Float[torch.Tensor, "seq_len seq_len"] = torch.triu(-1 * torch.inf * torch.ones(n_c, n_c), diagonal=1)
        return mask

    def forward(self, x: Float[torch.Tensor, "seq_len d_model"]) -> Float[torch.Tensor, "seq_len d_model"]:
        #create mask, with size n_c x n_c
        mask = self.create_mask(x.shape[0])

        #compute attention scores
        # A = softmax((X @ W_qk @ X^T) + M) @ X @ W_vo
        A = self.softmax((self.W_qk(x)) @ x.transpose(0, -1) + mask) @ self.W_vo(x)
        return A

In [6]:
class TransformerBlock(torch.nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.attention_head = AttentionHead(config)
        self.mlp = MLP(config)

    def forward(self, x: Float[torch.Tensor, "seq_len d_model"]) -> Float[torch.Tensor, "seq_len d_model"]:
        return x + self.attention_head(x) + self.mlp(x)

In [7]:
class Transformer(torch.nn.Module):
    def __init__(self, num_blocks: int, config: Config):
        super().__init__()
        self.config = config
        self.embedding = nn.Linear(config.d_vocab, config.d_model)
        self.blocks = nn.ModuleList([TransformerBlock(config) for _ in range(num_blocks)])
        

    def forward(self, x: Float[torch.Tensor, "seq_len vocab"]) -> Float[torch.Tensor, "vocab seq_len"]:
        x = self.embedding(x)
        for block in self.blocks:
            x = block.forward(x)
        x = (x @ self.embedding.weight).T
        return x
    

In [ ]:
class Tokenizer():
    def __init__(self, raw_data: str):
        self.raw_data = raw_data 
        self.vocab_size = None
        self.vocab = None
        self.vocab_inverse = None
        
    def process_raw_data(self, text: str, 
                        allowed_punctuation: str = "-.,;:!?()\"" + "".join(str(x) for x in range(10)),
                        punctuation_convert: dict[str,str] = {'—': '-'}
                        ) -> str:
        for char, replacement in punctuation_convert.items():
            text = text.replace(char, replacement)
              
        text = '\n'.join(
                    line 
                    for line in text.split('\n')
                    if '.jpg' not in line
                )
        
        text = unicodedata.normalize('NFKD', text)

        # Encode to ASCII bytes, then decode back to string, ignoring errors
        text = text.encode('ascii', 'ignore').decode('ascii')

        # remove newlines and tabs
        text = text.replace('\n', ' ').replace('\t', ' ')

        for char in allowed_punctuation:
            text = text.replace(char, f' {char} ')
              
        text = text.strip()

        # remove multiple spaces
        while '  ' in text:
            text = text.replace('  ', ' ')

        text = ''.join((char if (char.isalnum() or char in allowed_punctuation or char == ' ') else ' ') for char in text)
        
        text = text.lower()

        text = text.strip()

        return text
    
    def tokenize(self, 
        text: str,
        process: bool = False,
    ):
        if process:
            text = self.process_raw_data(text)
        tokenized_text = text.split(' ')

        vocab_counts: Counter[str] = Counter(tokenized_text).most_common()
        self.vocab_size: int = len(vocab_counts)
        self.vocab: list[str] = [token for token, count in vocab_counts]
        self.vocab_inverse: dict[int: str] = {key: value for key, value in (self.vocab, range(0, self.vocab_size))}
        
    def encode(self, data: str) -> list[int]:
        return [self.vocab_inverse[word] for word in data]
    
    def decode(self, tokens: list[int]) -> str:
        return ''.join(self.vocab[token] for token in tokens)

# Tests

In [9]:
# Attention head test
x: Float[torch.Tensor, "seq_len d_model"] = torch.ones(5, 16)
config = Config(d_model=16, d_vocab=1000, d_hidden=64)
attention_head: AttentionHead = AttentionHead(config)
output: Float[torch.Tensor, "seq_len d_model"] = attention_head.forward(x)
print(output.shape)

torch.Size([5, 16])


In [10]:
# Test the whole thing
config = Config(d_model=16, d_vocab=1000, d_hidden=64)
transformer = Transformer(num_blocks=2, config=config)
x = torch.ones(config.d_vocab, dtype=torch.float)
y: Float[torch.Tensor, "vocab seq_len"] = transformer(x)
print(y.shape)
print(y)
print(x)

torch.Size([1000])
tensor([-5.5714e-02,  1.8208e-02,  2.2584e-03, -4.0544e-02,  2.2233e-03,
        -6.9042e-03,  2.7877e-02,  3.5115e-02, -1.4944e-02,  5.9888e-03,
        -6.9910e-02,  1.9059e-02,  7.3378e-03,  3.6847e-05, -1.4642e-02,
         5.9656e-02,  3.7088e-02,  2.3467e-03, -3.6977e-03, -1.9625e-02,
         1.7052e-02, -9.2979e-03, -6.0489e-02, -2.3581e-02, -3.8072e-02,
        -1.9473e-03,  5.2006e-02,  1.1954e-02, -7.5672e-02,  2.3997e-02,
        -2.6352e-02,  2.0462e-03,  3.5036e-02,  1.8057e-03,  5.6882e-02,
         7.0652e-02,  1.5618e-02, -9.6903e-03, -4.2127e-04,  4.1136e-02,
         7.8641e-03, -2.6780e-03,  3.3360e-02,  3.0056e-02, -1.5967e-02,
        -3.3927e-02, -8.1995e-02, -6.5845e-02,  1.6618e-02,  1.2403e-02,
        -1.3927e-02, -5.3685e-02, -2.7333e-02, -5.1440e-02, -7.7847e-02,
         1.8380e-02, -1.6425e-02,  2.2449e-02, -1.8473e-02, -3.5723e-02,
        -5.1367e-02, -2.6521e-02,  1.4641e-02, -2.1195e-02, -1.5389e-02,
         1.1004e-02, -1.6729e-02

C:\Users\Nathan\AppData\Local\Temp\ipykernel_136360\1079021095.py:13: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4419.)
  x = (x @ self.embedding.weight).T


# Training Loop

In [11]:
def train_model(
    model: Transformer,
    loss: torch.nn.CrossEntropyLoss = nn.CrossEntropyLoss(),
    lr: Float = 1e-3,
    epochs: Int = 1
    ):
    optimizer: torch.optim.SGD = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    string_data = get_many_books(ids=range(10,15), data_temp="./data/gutenberg_data")
    tokenized_string_data = [x for i in string_data[]]
    labels = {}
    int_labels = []
    for i in range(len(string_data)):
        labels[i] = string_data[i]
        int_labels.append(i)
    
    training_data = torch.tensor(int_labels)
    
    for epoch in range(epochs):
        
        optimizer.zero_grad()
        outputs = model(training_data)
    



SyntaxError: invalid syntax (341824798.py, line 9)

In [ ]:
config: Config = Config(d_model=16, d_vocab=1000, d_hidden=64)
model = Transformer(num_blocks=2, config=config)
train_model(model)

Getting book 10...
	4432261 characters read
Getting book 11...
	148062 characters read
Getting book 12...
	168390 characters read
Getting book 13...
	34579 characters read
Getting book 14...
	1951150 characters read


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float